# <center>Weight Data

Objective of this notebook is to bring in all the data related to my health:

1. Weighing scale data
2. MyFitness Pal
3. Garmin watch
4. Samsung Health 

In [1]:
import pandas as pd
import numpy as np
import os
import requests

os.chdir('/Users/jayrajparmar/Documents/side_project/health_data_tracking')

import basic_methods as bm

/usr/local/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
df_initial = pd.read_csv('/Users/jayrajparmar/Documents/side_project/health_data_tracking/Weight_data/Measurement Data.csv')
df_initial.head()

,Time of Measurement,Weight(kg),BMI,Body Fat(%),Fat-free Body Weight(kg),Subcutaneous Fat(%),Visceral Fat,Body Water(%),Skeletal Muscle(%),Muscle Mass(kg),Bone Mass(kg),Protein(%),BMR(kcal),Metabolic Age,Remarks
0,"April 9, 2022 09:28:03",80.9000,28.3000,16.01,67.95,13.1,10.0,60.6,54.3,64.50,3.40,19.1,1837.0,28.0,--
1,"April 8, 2022 06:19:05",81.6500,28.6000,16.19,68.43,13.2,10.0,60.5,54.1,65.00,3.42,19.1,1848.0,29.0,--
2,"April 7, 2022 06:08:45",81.6500,28.6000,16.22,68.41,13.2,10.0,60.5,54.1,65.00,3.42,19.1,1847.0,29.0,--
3,"April 6, 2022 06:14:36",81.2500,28.4000,16.08,68.18,13.1,10.0,60.6,54.2,64.80,3.41,19.1,1842.0,28.0,--
4,"April 5, 2022 06:04:17",81.4000,28.5000,16.13,68.27,13.2,10.0,60.6,54.2,64.90,3.41,19.1,1844.0,29.0,--


In [ ]:
bm.datetime_conversion(df_initial, 'Time of Measurement')

def datetime_conversion(df,col_list):
    df[col_list] = df[col_list].apply(pd.to_datetime, errors = 'coerce')
    

In [8]:
def data_prep(df):
    # Column name cleaning
    df.columns = [i.replace(' ','_').replace('(','').replace(')','') for i in list(df_initial)]
    print(df.columns)
    df = bm.datetime_conversion(df, 'Time_of_Measurement')
    return df
    

In [9]:
df_test = data_prep(df_initial)
df_test

Index(['Time_of_Measurement', 'Weightkg', 'BMI', 'Body_Fat%',
       'Fat-free_Body_Weightkg', 'Subcutaneous_Fat%', 'Visceral_Fat',
       'Body_Water%', 'Skeletal_Muscle%', 'Muscle_Masskg', 'Bone_Masskg',
       'Protein%', 'BMRkcal', 'Metabolic_Age', 'Remarks'],
      dtype='object')


In [149]:
# Column name cleaning
df_initial.columns = [i.replace(' ','_').replace('(','').replace(')','') for i in list(df_initial)]

# Getting index as the date from time of Measurement column

df_initial['Time_of_Measurement'] = pd.to_datetime(df_initial.Time_of_Measurement)
df_initial.index = df_initial['Time_of_Measurement'].dt.date

# Converting all the columns to numeric 
lis = []
for i in df_initial.columns:
    if df_initial[i].dtypes != 'datetime64[ns]':
        lis.append(df_initial[i].apply(pd.to_numeric, errors='coerce'))
        print(str(i)+str('(')+str(df_initial[i].dtypes)+str(')')+str(':')+'Done')
        
    else: print('nothing')
df_dtyp = pd.concat(lis, axis=1)
df_dtyp = df_dtyp.drop_duplicates()
del df_dtyp['Remarks']
df_dtyp = df_dtyp.dropna(axis='index')
df_initial.index.names = ['Dates']
df_final = df_dtyp.merge(pd.DataFrame(df_initial.groupby(['Dates'])['Time_of_Measurement'].max()),
                         how='left',left_index=True, right_index=True)
cols = ['Weightkg', 'BMI', 'Body_Fat%', 'Fat-free_Body_Weightkg',
       'Subcutaneous_Fat%', 'Visceral_Fat', 'Body_Water%', 'Skeletal_Muscle%',
       'Muscle_Masskg', 'Bone_Masskg', 'Protein%', 'BMRkcal', 'Metabolic_Age']
agg_func = ['max']*13

d = dict(zip(cols,agg_func))
df_dedup = df_final.groupby(['Time_of_Measurement']).agg(d)
df_dedup['Dates'] = df_dedup.index.date
print('*****************************Datatypes after conversion*****************************')
print(df_final.dtypes)
# df_final = df_dup.dropna(axis='index')

print(df_dtyp.shape)
print(df_final.shape)
print(df_dedup.shape)
# Generating 7 days rolling average for all the health indicators








nothing
Weightkg(float64):Done
BMI(float64):Done
Body_Fat%(object):Done
Fat-free_Body_Weightkg(object):Done
Subcutaneous_Fat%(object):Done
Visceral_Fat(object):Done
Body_Water%(object):Done
Skeletal_Muscle%(object):Done
Muscle_Masskg(object):Done
Bone_Masskg(object):Done
Protein%(object):Done
BMRkcal(object):Done
Metabolic_Age(object):Done
Remarks(object):Done
*****************************Datatypes after conversion*****************************
Weightkg                         float64
BMI                              float64
Body_Fat%                        float64
Fat-free_Body_Weightkg           float64
Subcutaneous_Fat%                float64
Visceral_Fat                     float64
Body_Water%                      float64
Skeletal_Muscle%                 float64
Muscle_Masskg                    float64
Bone_Masskg                      float64
Protein%                         float64
BMRkcal                          float64
Metabolic_Age                    float64
Time_of_Measurement